In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import dgl
from dgl.nn import ChebConv
import networkx as nx
import networkx.algorithms.community as nx_comm
import numpy as np
import pandas as pd
import numpy.linalg as alg
import os
from tqdm import tqdm
from functools import reduce
import argparse


def node_connects_cluster(node):
    return set(map(lambda x: inverse_cluster_dict[x], list(g[node]))).union(set([inverse_cluster_dict[node]]))


DGL backend not selected or invalid.  Assuming PyTorch for now.
/root/miniconda3/envs/myconda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [2]:

path = 'Dataset/socfb-Stanford3.mtx'

df = pd.read_table(path, skiprows=1, names = ["source", "target"], sep=" ")
g = nx.from_pandas_edgelist(df)

# calculate basic elements
num_nodes = g.number_of_nodes()
num_edges = g.number_of_edges()
degs = [g.degree[i] for i in g.nodes]
avg_deg = sum(degs)/len(degs)

res = 10


# clustering
# generally, we fix the outcome of clustering
clusters = nx_comm.louvain_communities(g, seed = 10, resolution=res)
clusters = sorted(clusters, key = len, reverse=True)
cluster_sizes = list(map(len, clusters))
num_cluster = len(clusters)

# dict: from node to its cluster
inverse_cluster_dict = {
    node: cl for cl in range(num_cluster) for node in clusters[cl]
}

# dict: from node to its connected cluster
node_to_connected_clusters = {
    node: node_connects_cluster(node) for node in range(1, num_nodes + 1)
}


for i in g.nodes:
    g.nodes[i]["deg"] = g.degree[i]

rev_cluster_sizes = cluster_sizes[::-1]
rev_clusters = clusters[::-1]
rev_cluster_sizes = np.array(rev_cluster_sizes)


In [3]:
dgl_G = dgl.from_networkx(g)

In [4]:
ramps = [0.02, 0.05, 0.1, 0.25, 0.5]
repeat_num = 1000

In [5]:

class GCN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = dgl.nn.ChebConv(2, 16, 2)
        self.conv2 = dgl.nn.ChebConv(16, 16, 1)
        self.conv3 = dgl.nn.ChebConv(16, 1, 1)

        
    def forward(self, g, features):
        x = self.conv1(g, features)
        x = self.conv2(g, x)
        x = self.conv3(g, x)
        return x
        

In [6]:
def po_linear_model_sqrt_expo(graph, alpha=1, beta=1, sigma=0.1, gamma=1):    
    for i in graph.nodes:
        graph.nodes[i]["y"] = alpha + beta * graph.nodes[i]["z"] + sigma * np.random.normal() +  gamma * np.sqrt(sum([graph.nodes[ngbr]['z'] for ngbr in graph[i]])/graph.degree[i])

In [ ]:
# cluster-level cr

GATE_hat_list = np.zeros((repeat_num, len(ramps)))
     
for seed in tqdm(range(repeat_num)):    
    np.random.seed(seed)     
    rollout_index = np.random.uniform(0, 1, size=(num_cluster,))   
    
    for num_step in range(len(ramps)): 
        print(f"Seed: {seed}")
        torch.manual_seed(2)
        p_list = [0, ramps[num_step]]                
        g_feat_list = []
        y_list = []        
        
        for p in p_list:            
            tr_clusters = np.arange(num_cluster)[rollout_index<np.quantile(rollout_index, p)]
            nx.set_node_attributes(g, 0, "z")
            if len(tr_clusters) > 0:
                tr_units = reduce(lambda x, y: x.union(y), [clusters[i] for i in tr_clusters])                  
                nx.set_node_attributes(g, {unit:1 for unit in tr_units}, "z")
            
            po_linear_model_sqrt_expo(g, gamma = 1)            
            g_feat_list.append(torch.tensor([[g.nodes[n]['z'], g.nodes[n]['deg']] for n in g.nodes], dtype=torch.float))
            y_list.append(torch.tensor([[g.nodes[n]['y']] for n in g.nodes], dtype=torch.float).reshape(-1))
    
        # Instantiate the model and optimizer
        model = GCN()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.004)
        
        # Train the model
        for epoch in range(400):
            for i in range(len(p_list)):
                model.train()
                optimizer.zero_grad()
                out = model(dgl_G, g_feat_list[i])
                loss = F.mse_loss(out.squeeze(), y_list[i])  
                loss.backward()
                optimizer.step()
            
                if epoch % 50 == 0:
                    print(f'Epoch {epoch}, Treat Prop: {p_list[i]:.2f}, Loss: {loss:.4f}')
    
        print(loss.item())
        g_feat = g_feat_list[0]
    
        g_feat[:,0] = 0
        global_control_pred = model(dgl_G, g_feat).detach().numpy()
    
        g_feat[:,0] = 1
        global_treat_pred = model(dgl_G, g_feat).detach().numpy()
    
        GATE_hat = global_treat_pred.mean() - global_control_pred.mean()
    
        GATE_hat_list[seed, num_step] = GATE_hat
        
    torch.save(GATE_hat_list, "Result/SQRT_incre_2step_CR_cluster.pkl")


  0%|          | 0/1000 [00:00<?, ?it/s]

Seed: 0


/root/miniconda3/envs/myconda/lib/python3.10/site-packages/dgl/nn/pytorch/conv/chebconv.py:108: DGLWarning: lambda_max is not provided, using default value of 2.  Please use dgl.laplacian_lambda_max to compute the eigenvalues.
  dgl_warning(


Epoch 0, Treat Prop: 0.00, Loss: 8.1246
Epoch 0, Treat Prop: 0.02, Loss: 2.3456
Epoch 50, Treat Prop: 0.00, Loss: 0.1585
Epoch 50, Treat Prop: 0.02, Loss: 0.1976
Epoch 100, Treat Prop: 0.00, Loss: 0.0284
Epoch 100, Treat Prop: 0.02, Loss: 0.0572
Epoch 150, Treat Prop: 0.00, Loss: 0.0163
Epoch 150, Treat Prop: 0.02, Loss: 0.0368
Epoch 200, Treat Prop: 0.00, Loss: 0.0151
Epoch 200, Treat Prop: 0.02, Loss: 0.0307
Epoch 250, Treat Prop: 0.00, Loss: 0.0144
Epoch 250, Treat Prop: 0.02, Loss: 0.0260
Epoch 300, Treat Prop: 0.00, Loss: 0.0140
Epoch 300, Treat Prop: 0.02, Loss: 0.0227
Epoch 350, Treat Prop: 0.00, Loss: 0.0136
Epoch 350, Treat Prop: 0.02, Loss: 0.0209
0.020331785082817078
Seed: 0
Epoch 0, Treat Prop: 0.00, Loss: 8.1190
Epoch 0, Treat Prop: 0.05, Loss: 2.3427
Epoch 50, Treat Prop: 0.00, Loss: 0.1689
Epoch 50, Treat Prop: 0.05, Loss: 0.2683
Epoch 100, Treat Prop: 0.00, Loss: 0.0361
Epoch 100, Treat Prop: 0.05, Loss: 0.1066
Epoch 150, Treat Prop: 0.00, Loss: 0.0230
Epoch 150, Treat 

  0%|          | 1/1000 [00:26<7:24:38, 26.71s/it]

...

0.056441184133291245
Seed: 193
Epoch 0, Treat Prop: 0.00, Loss: 8.1180
Epoch 0, Treat Prop: 0.02, Loss: 2.3543
Epoch 50, Treat Prop: 0.00, Loss: 0.1563
Epoch 50, Treat Prop: 0.02, Loss: 0.2005
Epoch 100, Treat Prop: 0.00, Loss: 0.0285
Epoch 100, Treat Prop: 0.02, Loss: 0.0598
Epoch 150, Treat Prop: 0.00, Loss: 0.0169
Epoch 150, Treat Prop: 0.02, Loss: 0.0387
Epoch 200, Treat Prop: 0.00, Loss: 0.0154
Epoch 200, Treat Prop: 0.02, Loss: 0.0320
Epoch 250, Treat Prop: 0.00, Loss: 0.0147
Epoch 250, Treat Prop: 0.02, Loss: 0.0264
Epoch 300, Treat Prop: 0.00, Loss: 0.0139
Epoch 300, Treat Prop: 0.02, Loss: 0.0244
Epoch 350, Treat Prop: 0.00, Loss: 0.0147
Epoch 350, Treat Prop: 0.02, Loss: 0.0226
0.02294289693236351
Seed: 193
Epoch 0, Treat Prop: 0.00, Loss: 8.1227
Epoch 0, Treat Prop: 0.05, Loss: 2.2888
Epoch 50, Treat Prop: 0.00, Loss: 0.1703
Epoch 50, Treat Prop: 0.05, Loss: 0.2668
Epoch 100, Treat Prop: 0.00, Loss: 0.0386
Epoch 100, Treat Prop: 0.05, Loss: 0.1020
Epoch 150, Treat Prop: 0.00

 19%|█▉        | 194/1000 [1:25:12<5:56:51, 26.56s/it]

0.07615821063518524
Seed: 194
Epoch 0, Treat Prop: 0.00, Loss: 8.1096
Epoch 0, Treat Prop: 0.02, Loss: 2.3301
Epoch 50, Treat Prop: 0.00, Loss: 0.1599
Epoch 50, Treat Prop: 0.02, Loss: 0.2037
Epoch 100, Treat Prop: 0.00, Loss: 0.0287
Epoch 100, Treat Prop: 0.02, Loss: 0.0641
Epoch 150, Treat Prop: 0.00, Loss: 0.0174
Epoch 150, Treat Prop: 0.02, Loss: 0.0427
Epoch 200, Treat Prop: 0.00, Loss: 0.0160
Epoch 200, Treat Prop: 0.02, Loss: 0.0354
Epoch 250, Treat Prop: 0.00, Loss: 0.0153
Epoch 250, Treat Prop: 0.02, Loss: 0.0306
Epoch 300, Treat Prop: 0.00, Loss: 0.0149
Epoch 300, Treat Prop: 0.02, Loss: 0.0271
Epoch 350, Treat Prop: 0.00, Loss: 0.0145
Epoch 350, Treat Prop: 0.02, Loss: 0.0247
0.02318902127444744
Seed: 194
Epoch 0, Treat Prop: 0.00, Loss: 8.1179
Epoch 0, Treat Prop: 0.05, Loss: 2.3505
Epoch 50, Treat Prop: 0.00, Loss: 0.1675
Epoch 50, Treat Prop: 0.05, Loss: 0.3078
Epoch 100, Treat Prop: 0.00, Loss: 0.0393
Epoch 100, Treat Prop: 0.05, Loss: 0.1293
Epoch 150, Treat Prop: 0.00,

 20%|█▉        | 195/1000 [1:25:40<5:59:22, 26.79s/it]

0.05974223464727402
Seed: 195
Epoch 0, Treat Prop: 0.00, Loss: 8.1154
Epoch 0, Treat Prop: 0.02, Loss: 2.3216
Epoch 50, Treat Prop: 0.00, Loss: 0.1575
Epoch 50, Treat Prop: 0.02, Loss: 0.2060
Epoch 100, Treat Prop: 0.00, Loss: 0.0302
Epoch 100, Treat Prop: 0.02, Loss: 0.0614
Epoch 150, Treat Prop: 0.00, Loss: 0.0174
Epoch 150, Treat Prop: 0.02, Loss: 0.0391
Epoch 200, Treat Prop: 0.00, Loss: 0.0161
Epoch 200, Treat Prop: 0.02, Loss: 0.0326
Epoch 250, Treat Prop: 0.00, Loss: 0.0156
Epoch 250, Treat Prop: 0.02, Loss: 0.0286
Epoch 300, Treat Prop: 0.00, Loss: 0.0152
Epoch 300, Treat Prop: 0.02, Loss: 0.0258
Epoch 350, Treat Prop: 0.00, Loss: 0.0147
Epoch 350, Treat Prop: 0.02, Loss: 0.0231
0.021739540621638298
Seed: 195
Epoch 0, Treat Prop: 0.00, Loss: 8.1115
Epoch 0, Treat Prop: 0.05, Loss: 2.3463
Epoch 50, Treat Prop: 0.00, Loss: 0.1661
Epoch 50, Treat Prop: 0.05, Loss: 0.2612
Epoch 100, Treat Prop: 0.00, Loss: 0.0346
Epoch 100, Treat Prop: 0.05, Loss: 0.1000
Epoch 150, Treat Prop: 0.00